# Import Databases

In [1]:
# Load libraries
import riptide
import cobra
import pandas as pd
import numpy as np

# Load & Format TPM File
#### Once files are made correctly, these steps do not need to be repeated. 
#### To re-run, uncomment lines with single "#"

In [ ]:
## Load TPM data
## Replace file path(s) accordingly
#df = pd.read_csv("/Users/kathleenfurtado/RIPTIDE/TPMs_PATRIC_ID.csv", sep=',')

In [ ]:
## Format IDs (remove 'fig|')
#df['PATRIC.ID'] = df['PATRIC.ID'].str.split('|', expand=True)[1]

In [ ]:
#df.shape 
## 3400 genes  
## need to filter down to just the metabolic genes present in the model

In [ ]:
## Get the genes from model
#modelGenes = {g.id for g in r20291.genes} #703 genes

In [ ]:
## Filter TPM df by modelGenes
#df1 = df.loc[df['PATRIC.ID'].isin(modelGenes)] # 678 genes (intersection of modelGenes and TPM genes)

In [ ]:
## Separate TPM file into condition-specific files
#df1.iloc[:, :4].to_csv('TPM_CDMM.csv',sep=',',index=False) # CDMM; isolates first three columns + row names
#df1.iloc[:, [0,4,5,6]].to_csv('TPM_Comm.csv',sep=',',index=False) # Comm; isolates columns 4-6 + row names
#df1.iloc[:, [0,7,8,9]].to_csv('TPM_IEC.csv',sep=',',index=False) # IEC; isolates columns 7-9 + row names

# Make Transcriptome Dictionaries

In [2]:
# Create transcriptomic dictionaries
cdmm = riptide.read_transcription_file('TPM_CDMM.csv', norm = False, header = True, sep=',')#TPM normalized, hence norm=False
iec = riptide.read_transcription_file('TPM_IEC.csv', norm = False, header = True, sep=',')

# Analyses for Complete CDMM

## Load Model for Complete CDMM (No Mucus)

In [4]:
##Media formulation for CDMM with all amino acids

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703.sbml")

cdmm_med = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00027_e', # Glucose
        'cpd00013_e', # Ammonia
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00054_e', # L-Serine
        'cpd00023_e', # L-Glutamate
        'cpd00033_e', # L-Glycine
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e'} # L-Asparagine

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_med}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

74.3745987238828

## Run RIPTIDE for No Mucus

In [6]:
# Create Riptide object
rip_cdmm = riptide.contextualize(model=r20291, conservative=False, transcriptome=cdmm)
rip_cdmm_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=cdmm) #used in paper

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 18:00:21.591473. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.1101 ; p = 0.0577
Fraction = 0.4 | Rho = 0.0759 ; p = 0.1912
Fraction = 0.45 | Rho = 0.0754 ; p = 0.194
Fraction = 0.5 | Rho = 0.034 ; p = 0.5586
Fraction = 0.55 | Rho = 0.0895 ; p = 0.1117
Fraction = 0.6 | Rho = 0.0073 ; p = 0.8969
Fraction = 0.65 | Rho = 0.0336 ; p = 0.5495
Fraction = 0.7 | Rho = 0.0611 ; p = 0.2724
Fraction = 0.75 | Rho = 0.0614 ; p = 0.2698
Fraction = 0.8 | Rho = 0.0625 ; p = 0.2622
Fraction = 0.85 | Rho = 0.0227 ; p = 0.6838
Fraction = 0.9 | Rho = 0.0123 ; p = 0.8238
Testing local objective fractions to 0.35...
Fraction = 0.325 | Rho = 0.1117 ; p = 0.0541
Fraction = 0.375 | Rho = 0.0869 ; p = 0.1344

Context-specific metabolism fit with 0.325 of optimal objective flux

Reactions pruned to 298 from 1313 (77.3% change)
Metabolites pruned to 300 from 1243 (75.86% change)
Flux through the objective DECREASED to ~

In [7]:
#Export output from CDMM condition
riptide.save_output(riptide_obj=rip_cdmm, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_complete')
riptide.save_output(riptide_obj=rip_cdmm_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_max_complete')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 18:21:38.234462. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [4]:
print(rip_cdmm_max.flux_samples.shape)

NameError: name 'rip_cdmm_max' is not defined

## Load Model for Complete CDMM +Mucus

In [10]:
##Load model and edit for CDMM media with monosaccharides derived from N- and O-glycans
#Using all amino acids

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703.sbml")

cdmm_muc = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00054_e', # L-Serine
        'cpd00023_e', # L-Glutamate
        'cpd00033_e', # L-Glycine
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e', # L-Asparagine
        'cpd00122_e', # N-Acetyl-D-glucosamine
        'cpd27607_e', # N-Acetyl-D-galactosamine
        'cpd00232_e', # Neu5Ac
        'cpd00108_e', # Galactose
        'cpd00751_e', # L-Fucose
        'cpd00138_e'} # D-Mannose

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_muc}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 18:34:27.883436. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


84.62195562073408

## Run RIPTIDE for  +Mucus

In [11]:
#Create RIPTIDE Objects
rip_iec = riptide.contextualize(model=r20291, conservative=False, transcriptome=iec)
rip_iec_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=iec)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 18:35:36.051312. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 341 from 1313 (74.03% change)
Metabolites pruned to 336 from 1243 (72.97% change)
Flux through the objective DECREASED to ~72.03 from ~84.62 (14.88% change)
Context-specific metabolism does not correlate with transcriptome (r=0.004, n.s.)

RIPTiDe completed in, 1 minute and 14 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0363 ; p = 0.5214
Fraction = 0.4 | Rho = 0.0351 ; p = 0.5355
Fraction = 0.45 | Rho = 0.0009 ; p = 0.9867
Fraction = 0.5 | Rho = 0.036 ; p = 0.5116
Fraction = 0.55 | Rho = 0.0572 ; p = 0.2991
Fraction = 0.6 | Rho = 0.0002 ; p = 0.997
Fraction = 0.65 | Rho = 0.0055 ; p = 0.9192
Fraction = 0.7 | Rho = -0.001 ; p = 0.9851
Fraction = 0.75 | Rho = 0.0068 ; p = 0.9
Fraction = 0.8 | Rho = 0.0042 ;

In [12]:
#Save Outputs
riptide.save_output(riptide_obj=rip_iec, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_complete')
riptide.save_output(riptide_obj=rip_iec_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_max_complete')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:01:58.375317. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [13]:
#Check feasible samplings per condition
print(rip_iec.flux_samples.shape) #287 samplings, 324 rxns
print(rip_iec_max.flux_samples.shape) #500 samplings, 341 rxns

(500, 341)
(500, 335)


/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:02:21.827203. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Test Complete CDMM +Mucus formulation, to see if this alters biomass/feasible solutions for no mucus condition.

In [14]:
#Create RIPTIDE Objects
rip_cdmm_monosacc_test = riptide.contextualize(model=r20291, conservative=False, transcriptome=cdmm)
rip_cdmm_monosacc_test_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=cdmm)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:02:58.759128. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 342 from 1313 (73.95% change)
Metabolites pruned to 338 from 1243 (72.81% change)
Flux through the objective DECREASED to ~70.05 from ~84.62 (17.22% change)
Context-specific metabolism does not correlate with transcriptome (r=0.04, n.s.)

RIPTiDe completed in, 2 minutes and 6 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.1087 ; p = 0.0543
Fraction = 0.4 | Rho = 0.1022 ; p = 0.0705
Fraction = 0.45 | Rho = 0.0309 ; p = 0.5848
Fraction = 0.5 | Rho = 0.0993 ; p = 0.0703
Fraction = 0.55 | Rho = 0.0537 ; p = 0.3263
Fraction = 0.6 | Rho = 0.0737 ; p = 0.1752
Fraction = 0.65 | Rho = 0.0824 ; p = 0.1284
Fraction = 0.7 | Rho = 0.0731 ; p = 0.1776
Fraction = 0.75 | Rho = 0.0308 ; p = 0.5706
Fraction = 0.8 | Rho = 0.03

In [15]:
#Check feasible samplings
print(rip_cdmm_monosacc_test.flux_samples.shape) #85 samplings, 342 rxns
print(rip_cdmm_monosacc_test_max.flux_samples.shape) #500 samplings, 314 rxns

(85, 342)
(500, 314)


/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:38:16.542419. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [16]:
#Export to check biomass and compare to conditions with mucus in R
riptide.save_output(riptide_obj=rip_cdmm_monosacc_test, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_monosacc_test')
riptide.save_output(riptide_obj=rip_cdmm_monosacc_test_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_monosacc_test_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:44:10.758749. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


# Repeat Analyses -G -S (matching lab-tested conditions)

## Load model for CDMM -G -S

In [17]:
##Load model and edit for CDMM media (no mucus)
#Glycine and Serine removed

#Load model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703.sbml")

#Specify ingredients for CDMM and their ID in model
cdmm_no_gs = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00023_e', # L-Glutamate
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e'} # L-Asparagine


# Add 'EX_' to specify exchange reactions for minimal media components
cdmm_ex = {'EX_' + x for x in cdmm_no_gs}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 21:37:46.443125. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


73.8476411998919

## Run RIPTIDE for No Mucus -G -S

In [18]:
# Create Riptide object
rip_cdmm_no_gs = riptide.contextualize(model=r20291, conservative=False, transcriptome=cdmm)
rip_cdmm_no_gs_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=cdmm)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 21:37:56.960353. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 324 from 1313 (75.32% change)
Metabolites pruned to 325 from 1243 (73.85% change)
Flux through the objective DECREASED to ~58.68 from ~73.85 (20.54% change)
Context-specific metabolism does not correlate with transcriptome (r=0.063, n.s.)

RIPTiDe completed in, 1 minute and 49 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0985 ; p = 0.089
Fraction = 0.4 | Rho = 0.1014 ; p = 0.0801
Fraction = 0.45 | Rho = 0.1162 ; p = 0.0446
Fraction = 0.5 | Rho = 0.0432 ; p = 0.4577
Fraction = 0.55 | Rho = 0.0715 ; p = 0.2135
Fraction = 0.6 | Rho = 0.0073 ; p = 0.8969
Fraction = 0.65 | Rho = 0.0636 ; p = 0.2603
Fraction = 0.7 | Rho = 0.0611 ; p = 0.2724
Fraction = 0.75 | Rho = 0.0614 ; p = 0.2698
Fraction = 0.8 | Rho = 0.06

In [19]:
#Export output from CDMM condition
riptide.save_output(riptide_obj=rip_cdmm_no_gs, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_no_gs')
riptide.save_output(riptide_obj=rip_cdmm_no_gs_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_no_gs_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 22:08:42.674944. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Load model for CDMM +Mucus -G -S

In [20]:
##Load model and edit for CDMM media with monosaccharides derived from N- and O-glycans
#Glycine and Serine Removed

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703.sbml")

cdmm_muc_no_gs = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00023_e', # L-Glutamate
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e', # L-Asparagine
        'cpd00122_e', # N-Acetyl-D-glucosamine
        'cpd27607_e', # N-Acetyl-D-galactosamine
        'cpd00232_e', # Neu5Ac
        'cpd00108_e', # Galactose
        'cpd00751_e', # L-Fucose
        'cpd00138_e'} # D-Mannose

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_muc_no_gs}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 22:08:50.490744. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


84.5537694497864

## Run RIPTIDE for +Mucus -G -S

In [21]:
#Create RIPTIDE Objects
rip_iec_no_gs = riptide.contextualize(model=r20291, conservative=False, transcriptome=iec)
rip_iec_no_gs_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=iec)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 22:09:18.091979. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 331 from 1313 (74.79% change)
Metabolites pruned to 328 from 1243 (73.61% change)
Flux through the objective DECREASED to ~66.06 from ~84.55 (21.87% change)
Context-specific metabolism does not correlate with transcriptome (r=0.038, n.s.)

RIPTiDe completed in, 1 minute and 10 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0363 ; p = 0.5214
Fraction = 0.4 | Rho = 0.0351 ; p = 0.5355
Fraction = 0.45 | Rho = 0.0071 ; p = 0.9005
Fraction = 0.5 | Rho = 0.0535 ; p = 0.3316
Fraction = 0.55 | Rho = 0.0504 ; p = 0.3579
Fraction = 0.6 | Rho = 0.0087 ; p = 0.8727
Fraction = 0.65 | Rho = 0.0164 ; p = 0.7669
Fraction = 0.7 | Rho = 0.0235 ; p = 0.6699
Fraction = 0.75 | Rho = 0.0127 ; p = 0.8183
Fraction = 0.8 | Rho = 0.0

In [22]:
#Save Outputs
riptide.save_output(riptide_obj=rip_iec_no_gs, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_no_gs')
riptide.save_output(riptide_obj=rip_iec_no_gs_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_no_gs_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 23:38:14.309865. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [23]:
#Check how many feasible solutions per condition.
print(rip_cdmm_no_gs_max.flux_samples.shape) #287 samplings, 324 rxns
print(rip_iec_no_gs_max.flux_samples.shape) #500 samplings, 331 rxns

(500, 299)
(500, 335)


/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 23:38:28.808397. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


# Repeat Analyses -G

## Load model for CDMM -G

In [24]:
##Load model and edit for CDMM media (no mucus)
#Using all amino acids
#Glycine removed

#Load model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703.sbml")

#Specify ingredients for CDMM and their ID in model
cdmm_no_g = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00054_e', # L-Serine
        'cpd00023_e', # L-Glutamate
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e'} # L-Asparagine

# Add 'EX_' to specify exchange reactions for minimal media components
cdmm_ex = {'EX_' + x for x in cdmm_no_g}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 23:55:24.063216. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


74.3745987238828

## Run RIPTIDE for No Mucus -G

In [25]:
#Create RIPTIDE Objects
rip_cdmm_no_g = riptide.contextualize(model=r20291, conservative=False, transcriptome=cdmm)
rip_cdmm_no_g_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=cdmm)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 23:55:49.729612. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 324 from 1313 (75.32% change)
Metabolites pruned to 325 from 1243 (73.85% change)
Flux through the objective DECREASED to ~58.68 from ~74.37 (21.1% change)
Context-specific metabolism does not correlate with transcriptome (r=0.062, n.s.)

RIPTiDe completed in, 1 minute and 53 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.1101 ; p = 0.0577
Fraction = 0.4 | Rho = 0.0759 ; p = 0.1912
Fraction = 0.45 | Rho = 0.0754 ; p = 0.194
Fraction = 0.5 | Rho = 0.034 ; p = 0.5586
Fraction = 0.55 | Rho = 0.0897 ; p = 0.1111
Fraction = 0.6 | Rho = 0.0073 ; p = 0.8969
Fraction = 0.65 | Rho = 0.0336 ; p = 0.5495
Fraction = 0.7 | Rho = 0.0638 ; p = 0.2525
Fraction = 0.75 | Rho = 0.0641 ; p = 0.2495
Fraction = 0.8 | Rho = 0.0625

In [28]:
#Save Output
riptide.save_output(riptide_obj=rip_cdmm_no_g, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_no_g')
riptide.save_output(riptide_obj=rip_cdmm_no_g_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_no_g_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 10:26:39.322363. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Load model for CDMM +Mucus -G

In [29]:
##Load model and edit for CDMM media with monosaccharides derived from N- and O-glycans
#Removed glycine

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703.sbml")

cdmm_muc_no_g = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00054_e', # L-Serine
        'cpd00023_e', # L-Glutamate
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e', # L-Asparagine
        'cpd00122_e', # N-Acetyl-D-glucosamine
        'cpd27607_e', # N-Acetyl-D-galactosamine
        'cpd00232_e', # Neu5Ac
        'cpd00108_e', # Galactose
        'cpd00751_e', # L-Fucose
        'cpd00138_e'} # D-Mannose

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_muc_no_g}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 10:27:05.220775. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


84.62195562073408

## Run RIPTIDE for +Mucus -G

In [30]:
#Create RIPTIDE Objects
rip_iec_no_g = riptide.contextualize(model=r20291, conservative=False, transcriptome=iec)
rip_iec_no_g_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=iec)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 10:27:34.658915. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 341 from 1313 (74.03% change)
Metabolites pruned to 336 from 1243 (72.97% change)
Flux through the objective DECREASED to ~72.03 from ~84.62 (14.88% change)
Context-specific metabolism does not correlate with transcriptome (r=0.004, n.s.)

RIPTiDe completed in, 1 minute and 16 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0363 ; p = 0.5214
Fraction = 0.4 | Rho = 0.0351 ; p = 0.5355
Fraction = 0.45 | Rho = 0.0009 ; p = 0.9867
Fraction = 0.5 | Rho = 0.036 ; p = 0.5116
Fraction = 0.55 | Rho = 0.0534 ; p = 0.3321
Fraction = 0.6 | Rho = 0.0002 ; p = 0.997
Fraction = 0.65 | Rho = 0.0055 ; p = 0.9192
Fraction = 0.7 | Rho = 0.0151 ; p = 0.7819
Fraction = 0.75 | Rho = 0.0068 ; p = 0.9
Fraction = 0.8 | Rho = 0.0042 ;

In [31]:
#Save Output
riptide.save_output(riptide_obj=rip_iec_no_g, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_no_g')
riptide.save_output(riptide_obj=rip_iec_no_g_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_no_g_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 10:50:12.865911. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [32]:
#Check how many feasible solutions per condition.
print(rip_cdmm_no_g_max.flux_samples.shape) #500 samplings, 298 rxns
print(rip_iec_no_g_max.flux_samples.shape) #500 samplings, 335 rxns

(500, 298)
(500, 335)


/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 10:50:25.551648. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


# Repeat Analyses -Ser

## Load model for CDMM -S

In [33]:
##Load model and edit for CDMM media (no mucus)
#Remove serine

#Load model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703.sbml")

#Specify ingredients for CDMM and their ID in model
cdmm_no_s = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00023_e', # L-Glutamate
        'cpd00033_e', # L-Glycine
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e'} # L-Asparagine

# Add 'EX_' to specify exchange reactions for minimal media components
cdmm_ex = {'EX_' + x for x in cdmm_no_s}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 10:51:52.705049. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


73.8476411998919

## Run RIPTIDE for No Mucus -S

In [34]:
#Create RIPTIDE Objects
rip_cdmm_no_s = riptide.contextualize(model=r20291, conservative=False, transcriptome=cdmm)
rip_cdmm_no_s_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=cdmm)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 10:51:58.766240. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 324 from 1313 (75.32% change)
Metabolites pruned to 325 from 1243 (73.85% change)
Flux through the objective DECREASED to ~58.68 from ~73.85 (20.54% change)
Context-specific metabolism does not correlate with transcriptome (r=0.062, n.s.)

RIPTiDe completed in, 1 minute and 55 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0985 ; p = 0.089
Fraction = 0.4 | Rho = 0.1014 ; p = 0.0801
Fraction = 0.45 | Rho = 0.1162 ; p = 0.0446
Fraction = 0.5 | Rho = 0.0432 ; p = 0.4577
Fraction = 0.55 | Rho = 0.0715 ; p = 0.2135
Fraction = 0.6 | Rho = 0.0073 ; p = 0.8969
Fraction = 0.65 | Rho = 0.0636 ; p = 0.2603
Fraction = 0.7 | Rho = 0.0611 ; p = 0.2724
Fraction = 0.75 | Rho = 0.0614 ; p = 0.2698
Fraction = 0.8 | Rho = 0.06

In [35]:
#Save Output
riptide.save_output(riptide_obj=rip_cdmm_no_s, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_no_s')
riptide.save_output(riptide_obj=rip_cdmm_no_s_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_no_s_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 11:16:29.654337. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Load model for CDMM +Mucus -S

In [36]:
##Load model and edit for CDMM media with monosaccharides derived from N- and O-glycans
#Serine removed

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703.sbml")

cdmm_muc_no_s = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00023_e', # L-Glutamate
        'cpd00033_e', # L-Glycine
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e', # L-Asparagine
        'cpd00122_e', # N-Acetyl-D-glucosamine
        'cpd27607_e', # N-Acetyl-D-galactosamine
        'cpd00232_e', # Neu5Ac
        'cpd00108_e', # Galactose
        'cpd00751_e', # L-Fucose
        'cpd00138_e'} # D-Mannose

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_muc_no_s}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 11:16:43.606115. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


84.5537694497864

## Run RIPTIDE for +Mucus -S

In [37]:
#Create RIPTIDE Objects
rip_iec_no_s = riptide.contextualize(model=r20291, conservative=False, transcriptome=iec)
rip_iec_no_s_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=iec)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 11:17:16.380461. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 331 from 1313 (74.79% change)
Metabolites pruned to 328 from 1243 (73.61% change)
Flux through the objective DECREASED to ~66.06 from ~84.55 (21.87% change)
Context-specific metabolism does not correlate with transcriptome (r=0.038, n.s.)

RIPTiDe completed in, 1 minute and 16 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0363 ; p = 0.5214
Fraction = 0.4 | Rho = 0.0381 ; p = 0.5016
Fraction = 0.45 | Rho = 0.0071 ; p = 0.9005
Fraction = 0.5 | Rho = 0.0535 ; p = 0.3316
Fraction = 0.55 | Rho = 0.0504 ; p = 0.358
Fraction = 0.6 | Rho = 0.0131 ; p = 0.8088
Fraction = 0.65 | Rho = 0.0164 ; p = 0.7669
Fraction = 0.7 | Rho = 0.0235 ; p = 0.6699
Fraction = 0.75 | Rho = 0.0127 ; p = 0.8183
Fraction = 0.8 | Rho = 0.03

In [38]:
#Save Output
riptide.save_output(riptide_obj=rip_iec_no_s, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_no_s')
riptide.save_output(riptide_obj=rip_iec_no_s_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_no_s_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 14:40:34.474075. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [39]:
#Check how many feasible solutions per condition.
print(rip_cdmm_no_s_max.flux_samples.shape) #500 samplings, 299 rxns
print(rip_iec_no_s_max.flux_samples.shape) #500 samplings, 335 rxns

(500, 299)
(500, 335)


/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-31 14:40:47.542318. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


# Run RIPTiDe for conditions -T

## Load model for CDMM -T

In [6]:
##Media formulation for CDMM with all amino acids

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703_gh.sbml")

cdmm_noT = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00027_e', # Glucose
        'cpd00013_e', # Ammonia
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00054_e', # L-Serine
        'cpd00023_e', # L-Glutamate
        'cpd00033_e', # L-Glycine
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e'} # L-Asparagine

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_noT}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

74.28140883863611

## Run RIPTIDE for CDMM -T

In [8]:
#Create RIPTIDE Objects
rip_cdmm_no_t = riptide.contextualize(model=r20291, conservative=False, transcriptome=cdmm)
rip_cdmm_no_t_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=cdmm)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 327 from 1313 (75.1% change)
Metabolites pruned to 326 from 1243 (73.77% change)
Flux through the objective DECREASED to ~65.69 from ~74.28 (11.56% change)
Context-specific metabolism does not correlate with transcriptome (r=0.014, n.s.)

RIPTiDe completed in, 2 minutes and 4 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0961 ; p = 0.0971
Fraction = 0.4 | Rho = 0.0686 ; p = 0.2372
Fraction = 0.45 | Rho = 0.0724 ; p = 0.212
Fraction = 0.5 | Rho = 0.0894 ; p = 0.1242
Fraction = 0.55 | Rho = 0.0484 ; p = 0.3892
Fraction = 0.6 | Rho = 0.0093 ; p = 0.8698
Fraction = 0.65 | Rho = 0.0645 ; p = 0.2494
Fraction = 0.7 | Rho = 0.0477 ; p = 0.3932
Fraction = 0.75 | Rho = 0.0241 ; p = 0.6646
Fraction = 0.8 | Rho = 0.014

## Load model for CDMM +Mucus -T

In [9]:
##Load model and edit for CDMM media with monosaccharides derived from N- and O-glycans
#Using all amino acids

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703_gh.sbml")

cdmm_muc_noT = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00054_e', # L-Serine
        'cpd00023_e', # L-Glutamate
        'cpd00033_e', # L-Glycine
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e', # L-Asparagine
        'cpd00122_e', # N-Acetyl-D-glucosamine
        'cpd27607_e', # N-Acetyl-D-galactosamine
        'cpd00232_e', # Neu5Ac
        'cpd00108_e', # Galactose
        'cpd00751_e', # L-Fucose
        'cpd00138_e'} # D-Mannose

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_muc_noT}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2024-01-04 18:41:04.366336. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


84.45587573354359

In [10]:
#Create RIPTIDE Objects
rip_iec_no_t = riptide.contextualize(model=r20291, conservative=False, transcriptome=iec)
rip_iec_no_t_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=iec)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2024-01-04 18:41:11.943385. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 343 from 1313 (73.88% change)
Metabolites pruned to 337 from 1243 (72.89% change)
Flux through the objective DECREASED to ~71.43 from ~84.46 (15.43% change)
Context-specific metabolism does not correlate with transcriptome (r=-0.036, n.s.)

RIPTiDe completed in, 1 minute and 13 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0118 ; p = 0.8342
Fraction = 0.4 | Rho = 0.0732 ; p = 0.1943
Fraction = 0.45 | Rho = -0.0191 ; p = 0.7345
Fraction = 0.5 | Rho = 0.0186 ; p = 0.7342
Fraction = 0.55 | Rho = 0.0366 ; p = 0.5048
Fraction = 0.6 | Rho = -0.0459 ; p = 0.3991
Fraction = 0.65 | Rho = -0.0122 ; p = 0.823
Fraction = 0.7 | Rho = -0.0049 ; p = 0.9286
Fraction = 0.75 | Rho = -0.037 ; p = 0.496
Fraction = 0.8 | Rho = 

In [11]:
#Save Output
riptide.save_output(riptide_obj=rip_cdmm_no_t, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_no_t')
riptide.save_output(riptide_obj=rip_cdmm_no_t_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_no_t_max')
riptide.save_output(riptide_obj=rip_iec_no_t, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_no_t')
riptide.save_output(riptide_obj=rip_iec_no_t_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_no_t_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2024-01-04 19:18:31.503612. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


### End of Analysis